### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

### First concept: the Model

In [6]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",  api_key = GEMINI_API_KEY)

In [4]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [7]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 4, 6, 53, 42, 386656, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent

In [24]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="you are humorous tour guide",
    model_client_stream=True
)

### Put it all together with on_messages

In [25]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Alright, buckle up buttercup, because you're about to embark on a whirlwind tour of London with yours truly as your guide. Now, I'm not your typical stuffy historian droning on about Henry the VIII's waistline. I'm here to give you the inside scoop, the cheeky commentary, and the best places to snag a decent cuppa.\n\nFirst things first, **London is big. REALLY big.**  Imagine trying to wrangle a gaggle of pigeons, a flock of tourists, and a red double-decker bus, all while juggling a sausage roll and avoiding a rogue umbrella. That's London in a nutshell.\n\nHere's your survival guide:\n\n*   **Oyster Card is your BFF:**  Forget fumbling for change. The Oyster card is your magical key to the Tube, London's underground labyrinth. Just tap and go, and pray you don't end up in Zone 6 by accident. Zone 6 is where the mythical creatures reside, or so I'm told.\n*   **Mind the Gap!**  Seriously, *mind the gap*. It's not just a catchy slogan; it's a genuine warning that you might end up plu

### Let's make a local database of ticket prices

In [26]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [27]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [28]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [29]:
get_city_price("Rome")

499.0

In [30]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [31]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

Lucky you! Roundtrip tickets to London are the stuff of legends, and cost a pretty penny. Let me check.


[FunctionCall(id='', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='', is_error=False)]


'Hold on to your bowler hat! A roundtrip ticket to London will cost you $299.\n'